In [261]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
# code is based on this resource - http://gabrielelanaro.github.io/blog/2016/03/03/decision-trees.html

def make_tree(x, y):
    '''
    x, y - pandas dataframes
    fields - np.ndarray
    '''
    x_fields = x.columns.values
    quant_fields = get_quantitives(x, x_fields)
    return recursive_split(x, y.values, quant_fields), quant_fields

def recursive_split(x, y, quantitive_fields):
    '''
    x - pandas dataframe
    '''
    
    # If there could be no split, just return the original set
    if is_pure(y) or len(y) == 0:
        return y

    # We get attribute that gives the highest information gain
    x_qualitative = [item for item in x.columns.values if item not in quantitive_fields]
    x_qualitative = x[x_qualitative]
    gain_qualitative = np.array([information_gain(y, x_attr) for x_attr in x_qualitative.values.T])
    gain_quantitive = [(get_ig_threshold(y, x[spec_field].values), spec_field) for spec_field in quantitive_fields]
    # print(gain_quantitive)
    
    gain = np.concatenate([gain_qualitative, np.array([item[0][0] for item in gain_quantitive])], axis=None) 
    
    selected_attr = np.argmax(gain)
    # print("Selected attr: {} and gain {}".format(selected_attr, gain[selected_attr]))
    if selected_attr >= len(gain_qualitative - 1):
        selected_attr = selected_attr - len(gain_qualitative) # Lol magic happens
        threshold = gain_quantitive[selected_attr][0][1]
        # print(threshold)
        selected_attr = quantitive_fields[selected_attr]
        # print(selected_attr)
        sets = partition(x[selected_attr].values, t=threshold)
        # print("DONE BY QUANTITIVE")
        # print(sets)
    else:
        selected_attr = [item for item in x.columns.values if item not in quantitive_fields][selected_attr]
        sets = partition(x[selected_attr].values)

    # If there's no gain at all, nothing has to be done, just return the original set
    if np.all(gain < 1e-6):
        return y
    
    
    # We split using the selected attribute
    
    res = {}
    for k, v in sets.items():
        # MAKE SUBSPLIT OF X
        y_subset = y.take(v, axis=0)
        x_subset = x.iloc[v]
        # print(k)

        res["{} = {}".format(selected_attr, k)] = recursive_split(
            x_subset, y_subset, quantitive_fields)

    return res

In [264]:
def partition(a, t=None):
    if t is not None:
        indexes = [i for i in range(len(a))]
        d = {}
        l1 = [i for i in range(len(a)) if a[i] < t]
        d['< {}'.format(t)] = l1
        d['>= {}'.format(t)] = [i for i in indexes if not i in l1]
        return d
    return {c: (a == c).nonzero()[0] for c in np.unique(a)}


def entropy(s):
    res = 0
    val, counts = np.unique(s, return_counts=True)
    freqs = counts.astype('float') / len(s)
    for p in freqs:
        if p != 0.0:
            res -= p * np.log2(p)
    return res


def entropy_border(y, x, t):
    bordered_indexes = [i for i in range(len(x)) if x[i] < t] 
    y_bordered_left = y.take(bordered_indexes)
    y_bordered_right = y.take([i for i in range(len(x)) if i not in bordered_indexes])

    p1 = len(bordered_indexes) * 1.0 / len(x)
    p2 = 1 - p1
    
    hh1 = entropy(y_bordered_left)
    hh2 = entropy(y_bordered_right)
    return p1 * hh1 + p2 * hh2
    

def get_quantitives(X, fields):
    """
    X - pandas dataframe
    """
    
    types = X.dtypes
    quantitives = []
    
    for field in fields:
        if types[field] == 'int64' or types[field] == 'float64':
            quantitives.append(field)
    
    return quantitives


def information_gain(y, x):
    res = entropy(y)
    # We partition x, according to attribute values x_i
    val, counts = np.unique(x, return_counts=True)
    freqs = counts.astype('float') / len(x)

    # We calculate a weighted average of the entropy
    for p, v in zip(freqs, val):
        res -= p * entropy(y[x == v])

    return res
        
def get_ig_threshold(y, x):
    '''
    split - np.array - quantitive predictor
    y - np.array - responses (binary)
    '''
    y_entropy = entropy(y)
    sorted_split = list(sorted(x))
    sorted_split, y = zip(*sorted(zip(x, y)))
    y = np.array(y)

    ts = []
    igs = []
    
    for i in range(len(x) - 1):
        t = (sorted_split[i] + sorted_split[i + 1]) / 2
        ig = y_entropy - entropy_border(y, sorted_split, t)
        ts.append(t)
        igs.append(ig)
    
    max_ig = max(igs)
    # print("MAX IG = {} its index = {} ".format(max_ig, igs.index(max_ig)))

    max_index = igs.index(max_ig)
    return max_ig, ts[max_index]
        
        
def is_pure(s):
    return len(set(s)) == 1


In [268]:
class Node:
    def __init__(self, answers):
        vals, counts = np.unique(answers, return_counts=True)
        self.answers = {}
        most_index = 0
        
        for i in range(len(vals)):
            if counts[most_index] < counts[i]:
                most_index = i
            self.answers[vals[i]] = counts[i]
            
        self.most_probable = vals[most_index]

    
    def probability(self, x):
        p = self.answers.get(x, None)
        if p is not None:
            return p
        return 0 # TODO: AAAAAAAAAAAAAA
    
    def predict(self, x):
        return self.most_probable

class Tree:
    def __init__(self, tree_obj, quant_fields):
        self.predictor = None
        self.quantitive = False
        self.connections = {}
        self.quant_fields = quant_fields
        self.tree = self.parse_dict(tree_obj)
        
    def parse_dict(self, dictionary):
        keys = dictionary.keys()
        splits = [key.split(" = ") for key in keys]
        self.predictor = splits[0][0]
        
        check = splits[0][1]
        for field in self.quant_fields:
            if field in self.predictor:
                self.qualitative: True
                break
        
        if len(keys) != len(splits):
            raise Error("How is it even possible?")
            
#         [['outlook', 'overcast'], ['outlook', 'rainy'], ['outlook', 'sunny']]
#         dict_keys(['outlook = overcast', 'outlook = rainy', 'outlook = sunny'])
            
        keys = list(keys)
        for i in range(len(keys)):
            if type(dictionary[keys[i]]) is dict:
                self.connections[splits[i][1]] = Tree(dictionary[keys[i]], self.quant_fields)
            elif type(dictionary[keys[i]]) is np.ndarray:
                self.connections[splits[i][1]] = Node(dictionary[keys[i]])
                
    def predict(self, x):
        """
        x - pandas series object
        """
        print(self.predictor)
        
        value = x[self.predictor]
        if self.quantitive:
            keys = self.connections.keys()
            t = keys[0][len('< '):]
            conjured = float(t)
            print("I found tee")
            print(t)
            if value < t:
                subnode = self.connections[0]
            else:
                subnode = self.connections[1]
        else:
            value = str(value)
            subnode = self.connections[value]
        return subnode.predict(x)
        

In [262]:
def information_gain_ratio(y, x):
    return information_gain(y, x) / intristic_value(x)

def intristic_value(x): # TODO: I DONT USE IT!
    """
    x - set of values of feature X
    """
    uniques, counts = np.unique(x, return_counts=True) # {Ti, ..., Tk} and {|Ti|, ... , |Tk|}
    val = 0
    for count in counts:
        if count == 0:
            continue
        ratio = count / sum(counts)
        val += ratio * np.log2(ratio)
    return -val

def predict(tree, x, quant_fields):
    """
    tree - constructed decision tree
    x - pandas series object -- <class 'pandas.core.series.Series'>
    """
    print(x)
    magic = Tree(tree, quant_fields)  # Not optimized
    result = magic.predict(x)
    print(result)
    

In [ ]:
item = train_set.iloc[0, :]
print(type(item))
item['outlook']

In [ ]:
'key > key'.split(' = ')

In [269]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

train_set = pd.read_csv('datasets/titanic_modified.csv')


X = train_set.iloc[:, :6]
X[['Embarked']] = X[['Embarked']].fillna(value='S')
X[['Age']] = X[['Age']].fillna(value=X[['Age']].mean())
X[['Pclass']] = X[['Pclass']].astype('object')
X[['SibSp']] = X[['SibSp']].astype('object')


y = train_set.iloc[:, 6]

train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0)

# fields = train_set.columns.values
tree, quant_fields = make_tree(train_x, train_y)
predicted = [predict(tree, train_x.iloc[i, :], quant_fields) for i in range(len(train_x))]

#test_y_list = test_y.list()
#diff = sum([predicted[i] != test_y_list[i] for i in range(len(test_y))])
#print(diff / len(test_y))

Pclass         3
Sex         male
Age         40.5
SibSp          0
Parch          2
Embarked       S
Name: 153, dtype: object
Sex
Pclass
Age


KeyError: '40.5'

In [ ]:
X['Age'].values

In [257]:
print(tree)

{'Sex = female': {'Pclass = 1': {'Age = < 8.0': array([0], dtype=int64), 'Age = >= 8.0': {'Parch = < 1.5': {'Age = < 49.5': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
      dtype=int64), 'Age = >= 49.5': {'Age = < 50.5': {'Parch = < 0.5': array([0], dtype=int64), 'Parch = >= 0.5': array([1], dtype=int64)}, 'Age = >= 50.5': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)}}, 'Parch = >= 1.5': {'SibSp = 0': array([1, 1, 1, 1, 1], dtype=int64), 'SibSp = 1': {'Age = < 30.5': {'Age = < 19.5': array([1], dtype=int64), 'Age = >= 19.5': array([0], dtype=int64)}, 'Age = >= 30.5': array([1], dtype=int64)}, 'SibSp = 2': array([1, 1], dtype=int64), 'SibSp = 3': array([1, 1], dtype=int64)}}}, 'Pclass = 2': {'Age = < 56.0': {'Age = < 23.5': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
      d